<h1>First part of assignment</h1>

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [13]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
table = soup.find_all('table')
df = pd.read_html(str(table))[0]

df = df[df.Borough != 'Not assigned']

In [8]:
df.style

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [11]:
print(df.shape)

(103, 3)


<h1>Second part of assignment</h1>

In [12]:
dfGeo=pd.read_csv("Geospatial_Coordinates.csv")
dfGeo.style

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [61]:
#listOfGeo=dfGeo["Postal Code"].to_list()
df[['Latitude','Longitude']]=0
for item in dfGeo["Postal Code"]:
    data=dfGeo[dfGeo["Postal Code"]==item]
    df.loc[df['Postal Code'] == item,['Latitude']]=data["Latitude"].values[0]
    df.loc[df['Postal Code'] == item,['Longitude']]=data["Longitude"].values[0]
    

  

In [62]:
df.style

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
9,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
11,M3B,North York,Don Mills,43.745906,-79.352188
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h1>Part 3 of the assignment</h1>

In [64]:
from sklearn.cluster import KMeans 

ImportError: dlopen(/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/__check_build/_check_build.cpython-37m-darwin.so, 2): Symbol not found: ____chkstk_darwin
  Referenced from: /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/__check_build/../.dylibs/libomp.dylib
  Expected in: /usr/lib/libSystem.B.dylib
 in /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/__check_build/../.dylibs/libomp.dylib
___________________________________________________________________________
Contents of /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/__check_build:
__init__.py               __pycache__               _check_build.cpython-37m-darwin.so
setup.py
___________________________________________________________________________
It seems that scikit-learn has not been built correctly.

If you have installed scikit-learn from source, please do not forget
to 